# 图像卷积
## 互相关运算
`corr2d`函数实现了二维卷积操作（即互相关运算），该函数接受输入张量`X`和卷积核张量`K`，并返回输出张量`Y`。

In [1]:
import torch
import torch.nn as nn

In [2]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w +1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    return Y

我们来[**验证上述二维互相关运算的输出**]。


In [7]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
X, K, corr2d(X, K)

(tensor([[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]]),
 tensor([[0., 1.],
         [2., 3.]]),
 tensor([[19., 25.],
         [37., 43.]]))

## 卷积层
卷积层对输入和卷积核进行互相关运算，并添加偏置，产生输出。所以，卷积层中需要被训练的参数为卷积核权重和标量偏置。就像我们之前随机初始化全连接层一样，在训练基于卷积层的模型时，我们也随机初始化卷积核权重。

基于上面定义的`corr2d`函数[**实现二维卷积层**]。在`__init__`构造函数中，将`weight`和`bias`声明为两个模型参数。前向传播函数调用`corr2d`函数并添加偏置。

In [8]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

 ## 卷积的一个应用：图像边缘检测
 如下是[**卷积层的一个简单应用：**]通过找到像素变化的位置，来(**检测图像中不同颜色的边缘**)。
首先，我们构造一个$6\times 8$像素的黑白图像。中间四列为黑色（$0$），其余像素为白色（$1$）。


In [9]:
X = torch.ones(6, 8)
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

接下来，我们构造一个高度为$1$、宽度为$2$的卷积核`K`。当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零。

In [11]:
K = torch.tensor([[1, -1]])

现在，我们对参数`X`（输入）和`K`（卷积核）执行互相关运算。
如下所示，[**输出`Y`中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘**]，其他情况的输出为$0$。

In [12]:
Y = corr2d(X, K)

In [14]:
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

**该卷积核只能检测垂直边缘，不能检测水平边缘**，若我们将X进行转置，在用K对其进行卷积，会发现并不能检测出边缘。

## 学习卷积核
如果我们只需寻找黑白边缘，那么以上`[1, -1]`的边缘检测器足以。然而，当有了更复杂数值的卷积核，或者连续的卷积层时，我们不可能手动设计滤波器。那么我们是否可以[**学习由`X`生成`Y`的卷积核**]呢？

现在让我们看看是否可以通过仅查看“输入-输出”对来学习由`X`生成`Y`的卷积核。
我们先构造一个卷积层，并将其卷积核初始化为随机张量。接下来，在每次迭代中，我们比较`Y`与卷积层输出的平方误差，然后计算梯度来更新卷积核。为了简单起见，我们在此使用内置的二维卷积层，并忽略偏置。

In [17]:
# 构造一个二维卷积层，它具有一个输入通道、一个输出通道和一个形状为（1,2）的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape(1, 1, 6, 8)
Y = Y.reshape(1, 1, 6, 7)
lr = 3e-2
num_epochs = 10

for epoch in range(num_epochs):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (epoch + 1) % 2 == 0:
        print(f'epoch {epoch + 1}, loss {l.sum():.3f}')

epoch 2, loss 12.303
epoch 4, loss 3.974
epoch 6, loss 1.449
epoch 8, loss 0.564
epoch 10, loss 0.226


In [19]:
conv2d.weight.data

tensor([[[[ 0.9404, -1.0378]]]])

In [20]:
Y_hat

tensor([[[[ 0.1217,  1.0438,  0.0000,  0.0000,  0.0000, -0.9221,  0.1217],
          [ 0.1217,  1.0438,  0.0000,  0.0000,  0.0000, -0.9221,  0.1217],
          [ 0.1217,  1.0438,  0.0000,  0.0000,  0.0000, -0.9221,  0.1217],
          [ 0.1217,  1.0438,  0.0000,  0.0000,  0.0000, -0.9221,  0.1217],
          [ 0.1217,  1.0438,  0.0000,  0.0000,  0.0000, -0.9221,  0.1217],
          [ 0.1217,  1.0438,  0.0000,  0.0000,  0.0000, -0.9221,  0.1217]]]],
       grad_fn=<ConvolutionBackward0>)

可以看出，经过10次迭代后，卷积核和边缘检测器的误差已经很小。